In [ ]:
import random
random.seed(42)

In [ ]:
!mkdir spotGEO

In [ ]:
!mkdir spotGEO/train

In [ ]:
!mkdir spotGEO/train/images

In [ ]:
!mkdir spotGEO/train/annotations

In [ ]:
!mkdir spotGEO/validation

In [ ]:
!mkdir spotGEO/validation/images

In [ ]:
!mkdir spotGEO/validation/annotations

In [ ]:
!mkdir spotGEO/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/spotGEO/train.zip' -d '/content/spotGEO/data/'

In [ ]:
import json

In [ ]:
with open('/content/drive/My Drive/spotGEO/train_anno.json') as annotations:
        annotations = json.load(annotations)

In [ ]:
from lxml import etree

In [ ]:
def make_annotations(write_path, start_seq_id, end_seq_id, radius):
  for seq_id in range(start_seq_id,end_seq_id+1):
    if (seq_id % 2) == 0:
      for frm_id in range (1,6):
        if (frm_id % 2) != 0:

            filename_ = 'seq_'+str(seq_id)+'_frm_'+str(frm_id)+'.jpg'
            path_ = write_path

            annotation = etree.Element('annotation')
            folder = etree.SubElement(annotation, 'folder')
            folder.text = 'images'
            filename = etree.SubElement(annotation, 'filename')
            filename.text = filename_
            path = etree.SubElement(annotation, 'path')
            path.text = path_+filename_
            source = etree.SubElement(annotation, 'source')
            database = etree.SubElement(source, 'database')
            database.text = 'Unknown'
            size = etree.SubElement(annotation, 'size')
            width = etree.SubElement(size, 'width')
            width.text = '640'
            height = etree.SubElement(size, 'height')
            height.text = '480'
            depth = etree.SubElement(size, 'depth')
            depth.text = '1'
            segmented = etree.SubElement(annotation, 'segmented')
            segmented.text = '0'

            if annotations[seq_id-1]['num_objects'] == 0:
              pass

            else:
              for i in range(0,len(annotations[seq_id-1]['object_coords'])):
                object = etree.SubElement(annotation, 'object')
                name = etree.SubElement(object, 'name')
                name.text = 'satellite'
                pose = etree.SubElement(object, 'pose')
                pose.text = 'Unspecified'
                truncated = etree.SubElement(object, 'truncated')
                truncated.text = '0'
                difficult = etree.SubElement(object, 'difficult')
                difficult.text = '1'
                bndbox = etree.SubElement(object, 'bndbox')
                xmin = etree.SubElement(bndbox, 'xmin')
                xmin.text = str(int(annotations[seq_id-1]['object_coords'][i][0] + .5) - radius)
                ymin = etree.SubElement(bndbox, 'ymin')
                ymin.text = str(int(annotations[seq_id-1]['object_coords'][i][1] + .5) - radius)
                xmax = etree.SubElement(bndbox, 'xmax')
                xmax.text = str(int(annotations[seq_id-1]['object_coords'][i][0] + .5) + radius + 1)
                ymax = etree.SubElement(bndbox, 'ymax')
                ymax.text = str(int(annotations[seq_id-1]['object_coords'][i][1] + .5) + radius + 1)

                et = etree.ElementTree(annotation)
                et.write(path_+filename_[:-3]+'xml', pretty_print=True)

        else: pass
    else: pass

In [ ]:
%%time

make_annotations('/content/spotGEO/train/annotations/', 1, 1000, 3)

In [ ]:
%%time

make_annotations('/content/spotGEO/validation/annotations/', 1001, 1280, 3)

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
images_with_satellites_train = [f for f in listdir('/content/spotGEO/train/annotations/') if isfile(join('/content/spotGEO/train/annotations/', f))]
len(images_with_satellites_train)

In [ ]:
images_with_satellites_validation = [f for f in listdir('/content/spotGEO/validation/annotations/') if isfile(join('/content/spotGEO/validation/annotations/', f))]
len(images_with_satellites_validation)

In [ ]:
images_with_satellites = images_with_satellites_train + images_with_satellites_validation

In [ ]:
from PIL import Image

In [ ]:
def copy_train_images(read_path, write_path, start_seq_id, end_seq_id):
  for seq_id in range(start_seq_id,end_seq_id+1):
    for frm_id in range(1,6):
      if 'seq_'+str(seq_id)+'_frm_'+str(frm_id)+'.xml' in images_with_satellites:
        im = Image.open(read_path+str(seq_id)+'/'+str(frm_id)+'.png')
        im.save(write_path+'seq_'+str(seq_id)+'_frm_'+str(frm_id)+'.jpg')
      else:
        pass

In [ ]:
%%time

copy_train_images('/content/spotGEO/data/train/', '/content/spotGEO/train/images/', 1, 1000)

In [ ]:
train_images = [f for f in listdir('/content/spotGEO/train/images/') if isfile(join('/content/spotGEO/train/images/', f))]
len(train_images)

In [ ]:
%%time

copy_train_images('/content/spotGEO/data/train/', '/content/spotGEO/validation/images/', 1001, 1280)

In [ ]:
validation_images = [f for f in listdir('/content/spotGEO/validation/images/') if isfile(join('/content/spotGEO/validation/images/', f))]
len(validation_images)

In [ ]:
!pip3 install tensorflow-gpu==1.13.1

In [ ]:
!pip3 install keras==2.2.4

In [ ]:
!pip3 install numpy==1.16.1

In [ ]:
!pip3 install imageai --upgrade

In [ ]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

In [ ]:
%tensorflow_version 1.x

In [ ]:
%%time

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory='spotGEO')
trainer.setTrainConfig(object_names_array=['satellite'], batch_size=8, num_experiments=5, train_from_pretrained_model='pretrained-yolov3.h5')
trainer.trainModel()

In [ ]:
!cp 'spotGE+O/json/detection_config.json' -d '/content/drive/My Drive/spotGEO/'

In [ ]:
from os import listdir
from os.path import isfile, join

model_path = 'spotGEO/models/'
onlyfiles = [f for f in listdir(model_path) if isfile(join(model_path, f))]
final_model = model_path+sorted(onlyfiles)[-1]

In [ ]:
!cp $final_model -d '/content/drive/My Drive/spotGEO/'

In [ ]:
%%time

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory='spotGEO')
trainer.evaluateModel(model_path='spotGEO/models', json_path='spotGEO/json/detection_config.json', iou_threshold=0.4, object_threshold=0.3, nms_threshold=0.5)

In [ ]:
%%time

from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory='spotGEO')
trainer.evaluateModel(model_path='spotGEO/models', json_path='spotGEO/json/detection_config.json', iou_threshold=0.1, object_threshold=0.1, nms_threshold=0.1)

In [ ]:
%%time

from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath('spotGEO/models/') 
detector.setJsonPath('spotGEO/json/detection_config.json')
detector.loadModel()

In [ ]:
%%time

detections = detector.detectObjectsFromImage(input_image='spotGEO/validation/images/seq_1062_frm_3.jpg', output_image_path='spotGEO/test.jpg', minimum_percentage_probability=30)
for detection in detections:
    print(detection['name'], ' : ', detection['percentage_probability'], ' : ', detection['box_points'])

In [ ]:
len(detections)